
## LIVE: Text Encodings
AppliedAICourse


### Agenda
Prerequsites:
- Chapter: "Real world problem: Predict rating given product reviews on Amazon".

Focus:
- Code-walkthrough for each of the methods.
- Data Loading
- Pre-processing using NLTK.
- BoW using SkLearn + Sparse Matrices
- TF-IDF using SkLearn
- Word2VEc using Gensim
- BERT using bert-serving library + APIs.




### Data loading

In [2]:
# We have seen how to mount GDrive in earlier sessions

# This session: download data from internet directly to this box and use it

# Source: http://ai.stanford.edu/~amaas/data/sentiment/
! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2021-12-17 11:22:01--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  19.8MB/s    in 5.3s    

2021-12-17 11:22:07 (15.2 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [3]:
# uncompress and see the data
! ls
! pwd

aclImdb_v1.tar.gz  sample_data
/content


In [4]:
# it gives an error since the given file format is .tar.gz

! unzip aclImdb_v1.tar.gz

Archive:  aclImdb_v1.tar.gz
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of aclImdb_v1.tar.gz or
        aclImdb_v1.tar.gz.zip, and cannot find aclImdb_v1.tar.gz.ZIP, period.


In [5]:
#Google: "Unzip tar gz file colab" ----> https://stackoverflow.com/questions/49685924/extract-google-drive-zip-from-google-colab-notebook
import shutil
shutil.unpack_archive("/content/aclImdb_v1.tar.gz", "/content/")


In [6]:
# it gives the contents in that content folder
!ls /content/

aclImdb  aclImdb_v1.tar.gz  sample_data


In [7]:
# to see the contents in aclImdb 

! ls -l /content/aclImdb

total 1724
-rw-r--r-- 1 7297 1000 903029 Jun 11  2011 imdbEr.txt
-rw-r--r-- 1 7297 1000 845980 Apr 12  2011 imdb.vocab
-rw-r--r-- 1 7297 1000   4037 Jun 26  2011 README
drwxr-xr-x 4 7297 1000   4096 Apr 12  2011 test
drwxr-xr-x 5 7297 1000   4096 Jun 26  2011 train


In [8]:
# to see top 10 line in the folder imdb.vocab

! head -10 /content/aclImdb/imdb.vocab

the
and
a
of
to
is
it
in
i
this


In [9]:
# read the description and look the descriptions of al the variables

! ls /content/aclImdb/train


labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [ ]:
! ls /content/aclImdb/train/pos

In [11]:
# we can use cat or head function to see what is in the file

! head /content/aclImdb/train/pos/6250_10.txt

There is something special about the Austrian movies not only by Seidl, but by Spielmann and other directors as well. This is the piercing sense of reality that never leaves the viewer throughout the movie. Hundstage is no exception. This effect is achieved not only by the depicted stories but also by actors playing. In Hundstage I have never had the feeling that these are actors playing, but real people instead. So real is the visceral feeling of the viewer...Almost as if the grumpy pensioner or lonely lady in the movie are living below you in your block.<br /><br />Any person living in Vienna can without any doubt painfully recognize the people in the movie with their meckern/sudern (complaining), their hidden sexual urges and the prolo macho guys. This is further reinforced by the Viennese dialect which is, according to many, especially made for complaining as a way of life. A special parochialism and arrogance typical for Vienna are also very well portrayed.<br /><br />The Viennese

In [ ]:
# load data from k-reviews from imdb/train/pos reviews
# same code as previous sessions [Dimensionality Reduction session]
# to append all the individual txt review to a list raw_data[]

k=100

raw_data = [] # empty list, 

# Question: Why list of strings? 
# ans- list accepts duplicates, mutable, indexable, iterable
# -most of the libraries accepts 'List' of strings datatype

index_file = dict(); # store mapping from index to filename

import os
directory = r'/content/aclImdb/train/pos/'

i=0

# os.listdir() lists all the files in the directory
for f in os.listdir(directory): # for each file in the subfolder
      
  if f.endswith(".txt"): # check for text file
    fname = directory + "/" + f
    
    tmp = open(fname, "r") # read file 

    raw_data.append(tmp.read())
    index_file[i] = fname
        
    i += 1

    if i==k: # read k files
      break


print(i)
print(index_file)
print(raw_data[0])



100
{0: '/content/aclImdb/train/pos//4406_9.txt', 1: '/content/aclImdb/train/pos//6099_8.txt', 2: '/content/aclImdb/train/pos//7858_10.txt', 3: '/content/aclImdb/train/pos//6088_10.txt', 4: '/content/aclImdb/train/pos//2888_10.txt', 5: '/content/aclImdb/train/pos//3970_7.txt', 6: '/content/aclImdb/train/pos//492_7.txt', 7: '/content/aclImdb/train/pos//2379_8.txt', 8: '/content/aclImdb/train/pos//4843_7.txt', 9: '/content/aclImdb/train/pos//2324_8.txt', 10: '/content/aclImdb/train/pos//2430_10.txt', 11: '/content/aclImdb/train/pos//10132_9.txt', 12: '/content/aclImdb/train/pos//8526_10.txt', 13: '/content/aclImdb/train/pos//4911_9.txt', 14: '/content/aclImdb/train/pos//10312_10.txt', 15: '/content/aclImdb/train/pos//10192_8.txt', 16: '/content/aclImdb/train/pos//197_9.txt', 17: '/content/aclImdb/train/pos//2028_10.txt', 18: '/content/aclImdb/train/pos//6585_10.txt', 19: '/content/aclImdb/train/pos//7959_10.txt', 20: '/content/aclImdb/train/pos//7958_10.txt', 21: '/content/aclImdb/train/

### Data preprocessing

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt') # for punctuations

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# 1. remove stop words + tokenize
# Google "stop word removal python". ----> Many libraries like NLTK, Spacy, Sklearn

# https://pythonprogramming.net/stop-words-nltk-tutorial/

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# sent is the file name that we have to give for preprocessing

example_sent = "This is a sample sentence, showing off the stop words filtration."
def stopWordRemoval(sent, stop_words):
  word_tokens = word_tokenize(sent) # tokenize
  filtered_sentence = "";
  
  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence += ' '+w   # we added space (' ') between words
  return filtered_sentence

stop_words = set(stopwords.words('english')) # NLTK 

print(raw_data[0])
print(stopWordRemoval(raw_data[0], stop_words) ) # call the function


As a huge baseball fan, my scrutiny of this film is how realistic it appears. Dennis Quaid had all of the right moves and stances of a major league pitcher. It is a fantastic true story told with just a little too much "Disney" for my taste.
 As huge baseball fan , scrutiny film realistic appears . Dennis Quaid right moves stances major league pitcher . It fantastic true story told little much `` Disney '' taste .


In [ ]:
# Lemmatize - uses language specific grammar rules (from plural senese to singular sense)
#            - lemmatization has higher time complexity than stemming
# stemming - weaker form of lemmatization but faster
# note- lemmatization is better than stemming
# Source: https://pythonprogramming.net/lemmatizing-nltk-tutorial/

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
 
lemmatizer = WordNetLemmatizer()

print(lemmatizer.lemmatize("cats"))
print(lemmatizer.lemmatize("cacti"))
print(lemmatizer.lemmatize("geese"))
print(lemmatizer.lemmatize("rocks"))
print(lemmatizer.lemmatize("python"))
print(lemmatizer.lemmatize("better", pos="a"))
print(lemmatizer.lemmatize("best", pos="a"))
print(lemmatizer.lemmatize("run"))
print(lemmatizer.lemmatize("run",'v'))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
cat
cactus
goose
rock
python
good
best
run
run


# Complete Preprocessing of NLP

In [ ]:
# in this cell we are removing stopwords , tokenisation, and lemmatization
 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

example_sent = "This is a sample sentence, showing off the stop words filtration."
def stopWord_Lemmatize(sent, stop_words, lemmatizer ):
  word_tokens = word_tokenize(sent) # tokenize
  return_sent = "";
  
  for w in word_tokens:
      if w not in stop_words:
          return_sent += " "+ lemmatizer.lemmatize(w)    # lemmatize w beofre adding it to the return_sent
  return return_sent

stop_words = set(stopwords.words('english')) # NLTK 
lemmatizer = WordNetLemmatizer()

print(raw_data[0])
print(stopWord_Lemmatize(raw_data[0], stop_words, lemmatizer) ) # call the function


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Strangely enough this movie never made it to the big screen in Denmark, so I had to wait for the video release. My expectations where high but they where in no way disappointed. As always with Ang Lee there is fantastic acting, an intelligent and thrilling plot that has you guessing right till the end and superb filming. Along with Unforgiven this is easily one of the two best westerns of the 90`s.<br /><br />People who expect something along the line of Mel Gibson in The Patriot(corny) or Braveheart(acceptable) will be sourly disappointed, all others who appreciate the above mentioned qualities will have a fantastic time watching it. 9 out of 10.
 Strang

In [ ]:

# pre-process the k sentences and store the result
processed_data = []

for sent in raw_data:
  processed_data.append(stopWord_Lemmatize(sent, stop_words, lemmatizer) )

### Bag-of-words (BoW)

In [ ]:
# Google "sklearn bag of words" ---> https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer() #https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
X_BoW = count_vect.fit_transform(processed_data)

# .get_feature_names() gives the feature names 

print(count_vect.get_feature_names()) 

In [ ]:
print(type(X_BoW))

<class 'scipy.sparse.csr.csr_matrix'>


In [ ]:
# Sparse representations vs dense Matrices
# note - in sparse matrixes, only non zero values will be stored so it reduces space
#Refer: https://docs.scipy.org/doc/scipy/reference/sparse.html

# .data.nbytes gives the information how many bytes file taken to store
print(X_BoW.data.nbytes) # Refer: https://stackoverflow.com/questions/43681279/why-is-scipy-sparse-matrix-memory-usage-indifferent-of-the-number-of-elements-in

# Convert X_BoW to dense - convert a sparse matrix to a dense matrix
# Ref: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.todense.html#scipy.sparse.csr_matrix.todense
X_BoW_Dense = X_BoW.todense();
print(X_BoW_Dense.data.nbytes)

# note - we obsersed the sparse matrix taken space 50 times less data than a dense matrix
# therefore, it is better to save data in sparse matrix than in dense matrix

print(X_BoW.shape)
print(X_BoW_Dense.shape)
# Always use sparse-matrix in these situations

84152
3557600
(100, 4447)
(100, 4447)


In [ ]:
print(X_BoW_Dense[0,:])
print(X_BoW_Dense[0,818])
print(X_BoW_Dense[0,817])

[[0 0 0 ... 0 0 0]]
0
0


### TD-IDF

In [ ]:
# Google: "TF-IDF SkLearn" ---> https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
# ngram_range(1,2) - means it will give both 1 gram and 2 grams, (1,1) - only unigrams, (2,2) - only bigrams 
# ngrams - combination of words - (1gram- 1 word, 2gram- 2 word)

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
X_tfidf = vectorizer.fit_transform(raw_data)

print(vectorizer.get_feature_names())

In [ ]:
print(type(X_tfidf))
print(X_tfidf.shape)
print(X_tfidf.data.nbytes) # Refer: https://stackoverflow.com/questions/43681279/why-is-scipy-sparse-matrix-memory-usage-indifferent-of-the-number-of-elements-in

X_tfidf_dense = X_tfidf.todense()
print(type(X_tfidf_dense))
print(X_tfidf_dense.shape)
print(X_tfidf_dense.data.nbytes)

<class 'scipy.sparse.csr.csr_matrix'>
(100, 14948)
158976
<class 'numpy.matrix'>
(100, 14948)
11958400


### Word2Vec

In [ ]:
# Gensim library
# Refer: https://machinelearningmastery.com/develop-word-embeddings-python-gensim/

# Download pretrained  vectors
# https://www.quora.com/How-can-I-download-the-Google-news-word2vec-pretrained-model-from-a-Ubuntu-terminal
! wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

# gunzip is for unzipping .gz files 
! gunzip GoogleNews-vectors-negative300.bin.gz   

--2021-11-12 16:31:45--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.29.238
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.29.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  76.4MB/s    in 21s     

2021-11-12 16:32:07 (74.5 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
! ls

aclImdb  aclImdb_v1.tar.gz  GoogleNews-vectors-negative300.bin	sample_data


In [ ]:
# takes time as the model is large to laod into RAM
# RAM consumption also shoots up. Kernel could restart and give you a larger RAM isnatnce like 25GB RAM

from gensim.models import KeyedVectors
filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [ ]:
result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('queen', 0.7118192911148071)]


In [ ]:
v = model['queen'];
print(type(v))
print(v.shape)

<class 'numpy.ndarray'>
(300,)


### BERT

- BERT vs Word2Vec
- BERT: Contextual encodings
- can be used in production but it requires high GPU so it becomes costly
- Note- BERT is bi-directional, it gives the values based on meaning of sentence i.e., it give different value for same word.
- Note- Bert is much better than w2vec -
-Issues- BERT is more computationally expensive but it doesn't take high RAM and w2vec takes high RAM

In [12]:
# There are many ways to obatin BERT encodings. We are using one of the 
# most simple+popular approaches

# In DL chpaters, we will see how to do it in Keras.

# use GPU based instance: Runtime---> Change Runtime --> GPU

# Chaneg to TensorFlow Version 1.x 
# at present tensorflow version 2 is present but it is not backward compatible meanscode for version1 will not work in version 2
%tensorflow_version 2.x

In [ ]:
#https://github.com/hanxiao/bert-as-service - it doesn't work on colab

# https://github.com/hanxiao/bert-as-service/issues/380 - works on colab
# Install BERT-SERVING client and Server
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

In [14]:
# dowload pretrained models
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

# unzip is for .zip files
!unzip uncased_L-12_H-768_A-12.zip

--2021-12-17 11:36:22--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.128, 142.250.157.128, 142.251.8.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   181MB/s    in 2.1s    

2021-12-17 11:36:24 (181 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [15]:
# Start BERT_SERVER on the current computer
!nohup bert-serving-start -model_dir=./uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [ ]:
# Use bert-client from python
# Takes time to execute as it uses GPU
from bert_serving.client import BertClient
bc = BertClient()
print (bc.encode(['First do it', 'then do it right', 'then do it better'])) # list of setences

In [ ]:
# give data in a list

v = bc.encode([raw_data[0]])

In [ ]:
print(type(v))
print(v.shape)


# To convert .ipynb file to .pdf file


In [ ]:
# first mount the google drive in files option


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/My\Drive/Colab_Notebooks

In [ ]:
pwd

'/content/drive/My Drive/Colab_Notebooks'

In [ ]:
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-generic-recommended


In [ ]:
#!jupyter nbconvert --to html pandas_basics_practice.ipynb
!jupyter nbconvert -- to pdf NLP_Text_Encodings.ipynb